In [7]:
using IJulia
using Manopt, Manifolds, Random, LinearAlgebra, ManifoldDiff
using ManifoldDiff: grad_distance, prox_distance
using DifferentialEquations, DiffEqCallbacks
using Plots, DataFrames
using DelimitedFiles

### Misc functions

In [8]:
function load_QAP_data(file_path::String)
    open(file_path, "r") do io
        # Read the integer (first line)
        n = parse(Int, readline(io))
        
        # Skip the empty line
        readline(io)
        
        # Read the first matrix (nxn)
        matrix1 = [parse.(Int, split(strip(readline(io)))) for _ in 1:n] |> x -> hcat(x...)
        
        # Skip the empty line
        readline(io)

        # Read the second matrix (nxn)
        matrix2 = [parse.(Int, split(strip(readline(io)))) for _ in 1:n] |> x -> hcat(x...)
        
        # Check if there is an empty line before the third matrix
        third_matrix = zeros(Int64, n, n)
        next_line = readline(io, keep=false)
        if next_line != ""
            # If the next line is not empty, it's part of the third matrix
            third_matrix = [parse.(Int, split(strip(next_line))) for _ in 1:n] |> x -> hcat(x...)
        end
        
        return (n, matrix1, matrix2, third_matrix)
    end
end

function cycle_graph_adjacency_matrix(n::Int, directed::Bool=true)
    """
    Generates the adjacency matrix of a cycle graph of length n.

    Args:
        n (Int): The number of nodes in the cycle graph.
        directed (Bool): If true, generates a directed cycle graph.
                         If false, generates an undirected cycle graph.

    Returns:
        Array{Int64, 2}: The adjacency matrix of the cycle graph.
    """
    # Initialize an n x n matrix with zeros
    adj_matrix = zeros(Int, n, n)
    
    if directed
        # Directed cycle graph
        for i in 1:(n-1)
            adj_matrix[i, i+1] = 1
        end
        adj_matrix[n, 1] = 1  # Last node connects to the first node
    else
        # Undirected cycle graph
        for i in 1:(n-1)
            adj_matrix[i, i+1] = 1
            adj_matrix[i+1, i] = 1
        end
        adj_matrix[n, 1] = 1
        adj_matrix[1, n] = 1  # Last node connects to the first node and vice versa
    end
    
    return adj_matrix
end

function load_cnf(file_name)
    c = Nothing
    open(file_name) do file
        for (idx, line) in enumerate(eachline(file))
            if idx == 1
                N = parse(Int32, split(line, " ")[3])
                M = parse(Int32, split(line, " ")[4])
                c = zeros(M,N)
            else
                variables = split(line, " ")
                for var_str in variables
                    var = parse(Int32, var_str)
                    if var != 0
                        if var > 0
                            c[idx-1, var] = 1
                        elseif var < 0
                            c[idx-1, -var] = -1
                        end
                    end
                end
            end
        end
    end
    return c
end

function satisfied(spin_config, c)
    function check_clause(row, state)
        for (index,elem) in enumerate(row)
            if elem == state[index]
                return true
            end
        end
    end

    incorrect_flag = false
    for clause in eachrow(c)
        if check_clause(clause, spin_config) != true
            incorrect_flag = true
            break
        end
    end

    if incorrect_flag
        return false
    end
    return true
end

function round_to_nearest_pm1(x)
    return x < 0.0 ? 0 : 1
end

function round_array_to_pm1(arr)
    return map(round_to_nearest_pm1, arr)
end

condition(u, t, integrator) = satisfied([s>0 ? 1 : -1 for s in u], c)
affect!(integrator) = terminate!(integrator)
CTDS_cb = DiscreteCallback(condition, affect!);

# The general quadratic assignment problem

The objective is to minimize the following objective function
$$
\min_{P\in \mathcal{P}_n} \text{tr}\left( A^T P^T B P + P^T C\right)
$$

### Loading a problem

In [14]:
#N, B, A, C = load_QAP_data("QAPs/my6.dat");
N, B, A, C = load_QAP_data("QAP_problems/nug12.dat");

#c = load_cnf("permutation_cnf/N"*string(N)*"s.cnf")
#M, N² = size(c)
#k = map(row -> count(x -> x != 0, row), eachrow(c))
#M₀ = OrthogonalMatrices(N);

In [15]:
cost_fnc(P) = tr(A'*P'*B*P+P'*C)

cost_fnc (generic function with 1 method)

### Using conventional solver

In [ ]:
using JuMP
using SCIP

function optimize_cost_function(A::Matrix{Int64}, B::Matrix{Int64}, C::Matrix{Int64}, N::Int)
    # Create a model with the SCIP solver
    model = Model(SCIP.Optimizer)

    # Define variables, each element of P can be 0 or 1 (boolean matrix)
    @variable(model, P[i=1:N, j=1:N], Bin)

    # Add constraints to ensure P is a permutation matrix
    @constraint(model, [i=1:N], sum(P[i, j] for j in 1:N) == 1)  # Each row sums to 1
    @constraint(model, [j=1:N], sum(P[i, j] for i in 1:N) == 1)  # Each column sums to 1

    # Define the objective function to minimize the cost function
    @objective(model, Min, tr(A'*P'*B*P+P'*C))

    # Solve the model
    optimize!(model)

    # Extract the solution
    optimal_P = value.(P)

    return optimal_P
end

optimal_P = optimize_cost_function(A, B, C, N)

println("Optimal P matrix:")
reshape(optimal_P, N, N)


Precompiling SCIP
  ✓ ASL_jll
  ✓ OpenBLAS32_jll
  ✓ METIS_jll
  ✓ bliss_jll
  ✓ Hwloc_jll
  ✓ Ncurses_jll
  ✓ boost_jll
  ✓ Readline_jll
  ✓ MUMPS_seq_jll
  ✓ SPRAL_jll
  ✓ Ipopt_jll
  ✓ SCIP_jll
  ✓ SCIP_PaPILO_jll
